# report

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os
import copy
import holidays
from datetime import date
from contract_select import contract_select
from hedge_calc import hedge_calc
from greek_calc import greek_calc, pnl_daily_calc
from position_adj_cost import position_adj_cost, initial_position_cost

# Set the first trading date

In [2]:
moneyness = 'out'
main_type = 'P'
hedge_type = 'P'
main_minDistance = 0.08
hedge_minDistance = 0.06
starting_date = '2018-12-10'

main_original = contract_select(select_type='main', date= starting_date, max_spread_pct = 0.05, spy_spx='spx', contract_type = main_type, minAsk=1.0, minDistance = main_minDistance, minDTE=15, moneyness=moneyness, num=2)
hedge_original = contract_select(select_type='hedge', date= starting_date, max_spread_pct = 0.05, spy_spx='spx', contract_type = hedge_type, minAsk=0.2, minDistance = hedge_minDistance, minDTE=15, moneyness=moneyness, num=3)
main_original.update(hedge_original)

position = hedge_calc(main_original)
initial_spread_cost = initial_position_cost(position)

# Ignore any holiday and friday

In [ ]:
greek_pnl_daily = pd.DataFrame()
greek_pnl_daily['daily_pnl'] = 0
greek_pnl_daily['adjust_cost'] = 0

d = copy.deepcopy(position)
daily = []

for i in range(30):
    
    greek_pnl_daily = greek_pnl_daily.append(greek_calc(d))
    d = pnl_daily_calc(d)
    trading_date = d['date']
    greek_pnl_daily['daily_pnl'][-1] =  d['daily_rtn']
    daily.append(d)
    #if the hedge contracts are all dropped since pnl_daily_calc cannot find the contracts in the next trading day
    if len(d['hedge']) == 0:
        date = get_next_trading_day(greek_pnl_daily.index[-1], 'spx')
        d['hedge'] = contract_select(select_type='hedge', date= date,  max_spread_pct = 0.05, spy_spx='spx', contract_type = hedge_type, minAsk=0.2, minDistance = hedge_minDistance, minDTE=15, moneyness=moneyness, num = 2)['hedge']
        d['hedge']['WEIGHT'] = 0
    #if the main contracts are all dropped since pnl_daily_calc cannot find the contracts in the next trading day
    if len(d['main']) == 0:
        date = get_next_trading_day(greek_pnl_daily.index[-1], 'spx')
        d['main'] = contract_select(select_type='main', date= date,  max_spread_pct = 0.05, spy_spx='spx', contract_type = main_type, minAsk=1.0, minDistance = main_minDistance, minDTE=15, moneyness=moneyness, num=3)['main']
        d['main']['WEIGHT'] = 0
        
    temp = copy.deepcopy(hedge_calc(d))
    
    # drop/wind up certain main contracts that meet some criteria
    date = temp['main'].index[0][0]
    if main_type == 'P': 
        temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] > temp['main']['[STRIKE]']]
        temp['main'] = temp['main'][temp['main']['[P_ASK]'] > 0.5]
    if main_type == 'C': 
        temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] < temp['main']['[STRIKE]']]
        temp['main'] = temp['main'][temp['main']['[C_ASK]'] > 0.5]
    temp['main'] = temp['main'][temp['main']['[DTE]'] > 5]
    temp['main'] = temp['main'][temp['main']['[STRIKE_DISTANCE_PCT]'] >= main_minDistance/2]
    if len(temp['main']) < 3:
        new_contract = contract_select(select_type='main', date = date, max_spread_pct = 0.05, spy_spx='spx', contract_type = main_type, minAsk=0.2, minDistance = main_minDistance, minDTE=15, moneyness=moneyness, num=2-len(temp['main']))
        temp['main'] = temp['main'].append(new_contract['main'])
        temp['main']['WEIGHT'] = -1/len(temp['main'])
    # drop/wind up certain hedge contracts that meet some criteria
    if hedge_type == 'P': temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] > temp['hedge']['[STRIKE]']]
    if hedge_type == 'C': temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] < temp['hedge']['[STRIKE]']]
    temp['hedge'] = temp['hedge'][temp['hedge']['[DTE]'] > 5]
    temp['hedge'] = temp['hedge'][temp['hedge']['[STRIKE_DISTANCE_PCT]'] >= hedge_minDistance/2]
    if len(temp['hedge']) < 3:
        new_contract = contract_select(select_type='hedge', date = date, max_spread_pct = 0.05, spy_spx='spx', contract_type = hedge_type, minAsk=0.2, minDistance = hedge_minDistance, minDTE=15, moneyness=moneyness, num=3-len(temp['hedge']))
        temp['hedge'] = temp['hedge'].append(new_contract['hedge'])
        temp['hedge']['WEIGHT'] = 1/len(temp['hedge'])
        
    greek_pnl_daily['adjust_cost'][-1] = position_adj_cost(d, temp)['adjust_cost']
    d=copy.deepcopy(temp)

# Wind up position every friday

In [ ]:
greek_pnl_daily = pd.DataFrame()
greek_pnl_daily['daily_pnl'] = 0
greek_pnl_daily['adjust_cost'] = 0

d = copy.deepcopy(position)
daily = []

for i in range(30):
    greek_pnl_daily = greek_pnl_daily.append(greek_calc(d))
    d = pnl_daily_calc(d)
    greek_pnl_daily['daily_pnl'][-1] =  d['daily_rtn']
    daily.append(d)
    #if the hedge contracts are all dropped since pnl_daily_calc cannot find the contracts in the next trading day
    if len(d['hedge']) == 0:
        date = get_next_trading_day(greek_pnl_daily.index[-1], 'spx')
        d['hedge'] = contract_select(select_type='hedge', date= date,  max_spread_pct = 0.05, spy_spx='spx', contract_type = hedge_type, minAsk=0.2, minDistance = hedge_minDistance, minDTE=15, moneyness=moneyness, num = 2)['hedge']
        d['hedge']['WEIGHT'] = 0
    #if the main contracts are all dropped since pnl_daily_calc cannot find the contracts in the next trading day
    if len(d['main']) == 0:
        date = get_next_trading_day(greek_pnl_daily.index[-1], 'spx')
        d['main'] = contract_select(select_type='main', date= date,  max_spread_pct = 0.05, spy_spx='spx', contract_type = main_type, minAsk=1.0, minDistance = main_minDistance, minDTE=15, moneyness=moneyness, num=3)['main']
        d['main']['WEIGHT'] = 0
        
    temp = copy.deepcopy(hedge_calc(d))
    
    # drop/wind up certain main contracts that meet some criteria
    date = temp['main'].index[0][0]
    if main_type == 'P': 
        temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] > temp['main']['[STRIKE]']]
        temp['main'] = temp['main'][temp['main']['[P_ASK]'] > 0.5]
    if main_type == 'C': 
        temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] < temp['main']['[STRIKE]']]
        temp['main'] = temp['main'][temp['main']['[C_ASK]'] > 0.5]
    temp['main'] = temp['main'][temp['main']['[DTE]'] > 5]
    temp['main'] = temp['main'][temp['main']['[STRIKE_DISTANCE_PCT]'] >= main_minDistance/2]
    if len(temp['main']) < 3:
        new_contract = contract_select(select_type='main', date = date, max_spread_pct = 0.05, spy_spx='spx', contract_type = main_type, minAsk=0.2, minDistance = main_minDistance, minDTE=15, moneyness=moneyness, num=2-len(temp['main']))
        temp['main'] = temp['main'].append(new_contract['main'])
        temp['main']['WEIGHT'] = -1/len(temp['main'])
    # drop/wind up certain hedge contracts that meet some criteria
    if hedge_type == 'P': temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] > temp['hedge']['[STRIKE]']]
    if hedge_type == 'C': temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] < temp['hedge']['[STRIKE]']]
    temp['hedge'] = temp['hedge'][temp['hedge']['[DTE]'] > 5]
    temp['hedge'] = temp['hedge'][temp['hedge']['[STRIKE_DISTANCE_PCT]'] >= hedge_minDistance/2]
    if len(temp['hedge']) < 3:
        new_contract = contract_select(select_type='hedge', date = date, max_spread_pct = 0.05, spy_spx='spx', contract_type = hedge_type, minAsk=0.2, minDistance = hedge_minDistance, minDTE=15, moneyness=moneyness, num=3-len(temp['hedge']))
        temp['hedge'] = temp['hedge'].append(new_contract['hedge'])
        temp['hedge']['WEIGHT'] = 1/len(temp['hedge'])
        
    greek_pnl_daily['adjust_cost'][-1] = position_adj_cost(d, temp)['adjust_cost']
    d=copy.deepcopy(temp)

# Wind up position every friday and day before any holiday

In [3]:
greek_pnl_daily = pd.DataFrame()
greek_pnl_daily['daily_pnl'] = 0
greek_pnl_daily['adjust_cost'] = 0

d = copy.deepcopy(position)
d['date'] = starting_date
daily = []

for i in range(2):
    greek_pnl_daily = greek_pnl_daily.append(greek_calc(d))
    d = pnl_daily_calc(d)
    greek_pnl_daily['daily_pnl'][-1] =  d['daily_rtn']
    daily.append(d)
    
    #if the hedge contracts are all dropped since pnl_daily_calc cannot find the contracts in the next trading day
    if len(d['hedge']) == 0:
        date = get_next_trading_day(greek_pnl_daily.index[-1], 'spx')
        d['hedge'] = contract_select(select_type='hedge', date= date,  max_spread_pct = 0.05, spy_spx='spx', contract_type = hedge_type, minAsk=0.2, minDistance = hedge_minDistance, minDTE=15, moneyness=moneyness, num = 2)['hedge']
        d['hedge']['WEIGHT'] = 0
        
    #if the main contracts are all dropped since pnl_daily_calc cannot find the contracts in the next trading day
    if len(d['main']) == 0:
        date = get_next_trading_day(greek_pnl_daily.index[-1], 'spx')
        d['main'] = contract_select(select_type='main', date= date,  max_spread_pct = 0.05, spy_spx='spx', contract_type = main_type, minAsk=1.0, minDistance = main_minDistance, minDTE=15, moneyness=moneyness, num=3)['main']
        d['main']['WEIGHT'] = 0
        
    temp = copy.deepcopy(hedge_calc(d))
    
    # drop/wind up certain main contracts that meet some criteria
    date = temp['main'].index[0][0]
    if main_type == 'P': 
        temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] > temp['main']['[STRIKE]']]
        temp['main'] = temp['main'][temp['main']['[P_ASK]'] > 0.5]
    if main_type == 'C': 
        temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] < temp['main']['[STRIKE]']]
        temp['main'] = temp['main'][temp['main']['[C_ASK]'] > 0.5]
    temp['main'] = temp['main'][temp['main']['[DTE]'] > 5]
    temp['main'] = temp['main'][temp['main']['[STRIKE_DISTANCE_PCT]'] >= main_minDistance/2]
    if len(temp['main']) < 3:
        new_contract = contract_select(select_type='main', date = date, max_spread_pct = 0.05, spy_spx='spx', contract_type = main_type, minAsk=0.2, minDistance = main_minDistance, minDTE=15, moneyness=moneyness, num=2-len(temp['main']))
        temp['main'] = temp['main'].append(new_contract['main'])
        temp['main']['WEIGHT'] = -1/len(temp['main'])
    # drop/wind up certain hedge contracts that meet some criteria
    if hedge_type == 'P': temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] > temp['hedge']['[STRIKE]']]
    if hedge_type == 'C': temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] < temp['hedge']['[STRIKE]']]
    temp['hedge'] = temp['hedge'][temp['hedge']['[DTE]'] > 5]
    temp['hedge'] = temp['hedge'][temp['hedge']['[STRIKE_DISTANCE_PCT]'] >= hedge_minDistance/2]
    if len(temp['hedge']) < 3:
        new_contract = contract_select(select_type='hedge', date = date, max_spread_pct = 0.05, spy_spx='spx', contract_type = hedge_type, minAsk=0.2, minDistance = hedge_minDistance, minDTE=15, moneyness=moneyness, num=3-len(temp['hedge']))
        temp['hedge'] = temp['hedge'].append(new_contract['hedge'])
        temp['hedge']['WEIGHT'] = 1/len(temp['hedge'])
        
    greek_pnl_daily['adjust_cost'][-1] = position_adj_cost(d, temp)['adjust_cost']
    d=copy.deepcopy(temp)

In [ ]:
greek_pnl_daily.to_clipboard()

In [ ]:
df = daily[-1]['main']
df

In [ ]:
greek_pnl_daily

In [ ]:
main_original

In [ ]:
d['main']['WEIGHT']

In [ ]:
nyse_holidays = holidays.financial_holidays('NYSE')

In [ ]:
'01/01/2000' in nyse_holidays

In [ ]:
date.weekday(date(2018,1,2))

In [7]:
date.fromisoformat(d['date'])

AttributeError: 'str' object has no attribute 'fromisoformat'